In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('./gemstone.csv')

In [3]:
df.head()

,Unnamed: 0,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [4]:
df=df.drop(labels=['Unnamed: 0','id'],axis=1)

In [5]:
X=df.drop(labels=['price'],axis=1)
Y=df[['price']]

In [6]:
X

,carat,cut,color,clarity,depth,table,x,y,z
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77
...,...,...,...,...,...,...,...,...,...
193568,0.31,Ideal,D,VVS2,61.1,56.0,4.35,4.39,2.67
193569,0.70,Premium,G,VVS2,60.3,58.0,5.75,5.77,3.47
193570,0.73,Very Good,F,SI1,63.1,57.0,5.72,5.75,3.62
193571,0.34,Very Good,D,SI1,62.9,55.0,4.45,4.49,2.81


In [7]:
Y

,price
0,13619
1,13387
2,2772
3,666
4,14453
...,...
193568,1130
193569,2874
193570,3036
193571,681


In [8]:
categorical_columns=X.select_dtypes(include='object').columns

In [9]:
numerical_columns=X.select_dtypes(exclude='object').columns

In [10]:
df['cut'].unique()
df['color'].unique()
df['clarity'].unique()

array(['VS2', 'SI2', 'VS1', 'SI1', 'IF', 'VVS2', 'VVS1', 'I1'],
      dtype=object)

In [11]:
cut_categories=['Fair','Good','Very Good','Premium','Ideal']
clarity_categories=['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']
color_categories=['D','E','F','G','H','I','J']

In [12]:
from sklearn.impute import SimpleImputer  ## For Missing Value
from sklearn.preprocessing import StandardScaler  ## Handling Feature Scaling
from sklearn.preprocessing import OrdinalEncoder ## Ordinal Encoding
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [13]:
numerical_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='median')),
        ('scaler',StandardScaler())
    ]
)

categorical_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='most_frequent')),
        ('ordinalEncoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
        ('scaler',StandardScaler())
    ]
)
        
preprocessor=ColumnTransformer([
    ('categorical_pipeline',categorical_pipeline,categorical_columns),
    ('numerical_pipeline',numerical_pipeline,numerical_columns)
])
    

In [14]:
numerical_pipeline

Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler())])

In [15]:
categorical_pipeline

Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                ('ordinalEncoder',
                 OrdinalEncoder(categories=[['Fair', 'Good', 'Very Good',
                                             'Premium', 'Ideal'],
                                            ['D', 'E', 'F', 'G', 'H', 'I', 'J'],
                                            ['I1', 'SI2', 'SI1', 'VS2', 'VS1',
                                             'VVS2', 'VVS1', 'IF']])),
                ('scaler', StandardScaler())])

In [16]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.3,random_state=42)

In [17]:
preprocessor.fit_transform(x_train)

array([[ 0.87410007, -0.93674681,  1.35074594, ..., -0.78045073,
        -0.83510329, -0.87602377],
       [-1.13764403,  0.91085333,  0.68445511, ...,  1.07322572,
         1.16638856,  0.94663261],
       [-0.13177198,  0.91085333,  0.01816428, ...,  1.70311578,
         1.75506264,  1.74223659],
       ...,
       [-0.13177198,  0.29498662,  0.01816428, ...,  0.99223986,
         0.92186241,  1.0478913 ],
       [-1.13764403,  0.29498662,  2.01703677, ..., -1.21237535,
        -1.19736425, -1.25212747],
       [-1.13764403,  0.29498662, -1.31441737, ..., -1.23937063,
        -1.2154773 , -1.20873089]])

In [18]:
x_train=pd.DataFrame(preprocessor.fit_transform(x_train),columns=preprocessor.get_feature_names_out())
x_test=pd.DataFrame(preprocessor.transform(x_test),columns=preprocessor.get_feature_names_out())

In [19]:
x_train.head()

,categorical_pipeline__cut,categorical_pipeline__color,categorical_pipeline__clarity,numerical_pipeline__carat,numerical_pipeline__depth,numerical_pipeline__table,numerical_pipeline__x,numerical_pipeline__y,numerical_pipeline__z
0,0.874100,-0.936747,1.350746,-0.823144,-1.129988,-0.641897,-0.780451,-0.835103,-0.876024
1,-1.137644,0.910853,0.684455,0.945023,-1.777823,0.921902,1.073226,1.166389,0.946633
2,-0.131772,0.910853,0.018164,1.958484,0.165682,0.400636,1.703116,1.755063,1.742237
3,0.874100,-0.320880,2.017037,-0.995648,-0.574701,-0.641897,-1.122391,-1.161138,-1.165334
4,-1.137644,1.526720,-0.648127,-0.995648,0.258230,0.400636,-1.176382,-1.152082,-1.136403


In [20]:
x_test.head()

,categorical_pipeline__cut,categorical_pipeline__color,categorical_pipeline__clarity,numerical_pipeline__carat,numerical_pipeline__depth,numerical_pipeline__table,numerical_pipeline__x,numerical_pipeline__y,numerical_pipeline__z
0,0.874100,-1.552614,-0.648127,-0.629077,0.258230,-0.120630,-0.600482,-0.581521,-0.572248
1,-1.137644,0.294987,-1.314417,2.605374,-2.148014,-0.120630,2.126042,2.198832,1.959219
2,-0.131772,-0.936747,2.017037,-1.125026,-1.222536,0.921902,-1.374347,-1.414721,-1.469110
3,-0.131772,1.526720,2.017037,-1.017211,-0.574701,0.921902,-1.158385,-1.161138,-1.194265
4,0.874100,0.910853,-0.648127,0.858771,0.628421,-0.641897,0.947248,0.985258,1.004495


In [21]:
from sklearn.linear_model import LinearRegression,Lasso,ElasticNet,Ridge
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [22]:
regression=LinearRegression()
regression.fit(x_train,y_train)

LinearRegression()

In [23]:
regression.coef_

array([[   68.80035873,  -464.67990411,   652.10059539,  6432.97591819,
         -132.34206204,   -70.48787525, -1701.38593925,  -494.17005097,
          -76.32351645]])

In [24]:
regression.intercept_

array([3976.8787389])

In [25]:
import numpy as np

def evalute_model(true,predicted):
    mae=mean_absolute_error(true,predicted)
    mse=mean_squared_error(true,predicted)
    rmse=np.sqrt(mean_squared_error(true,predicted))
    r2_squqre=r2_score(true,predicted)
    return mae,rmse,r2_squqre

In [27]:
models={
    "LinearRegression":LinearRegression(),
    "Lasso":Lasso(),
    "Ridge":Ridge(),
    "ElasticNet":ElasticNet()
}

model_list=[]
trained_model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(x_train,y_train)
    
    
    y_pred=model.predict(x_test)
    
    
    mae,rmse,r2_squqre=evalute_model(y_test,y_pred)
    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
        
    print('Model Trainning Performance')
    print('rmse',rmse)
    print('MAE',mae)
    print('r2_score',r2_score)
    
    r2_list.append(r2_squqre)
    
    print('='*35)

LinearRegression
Model Trainning Performance
rmse 1014.6296630375477
MAE 675.0758270067404
r2_score <function r2_score at 0x0000022E9C973820>
Lasso
Model Trainning Performance
rmse 1014.6591018136338
MAE 676.2413837901756
r2_score <function r2_score at 0x0000022E9C973820>
Ridge
Model Trainning Performance
rmse 1014.6343233534418
MAE 675.1077629781364
r2_score <function r2_score at 0x0000022E9C973820>
ElasticNet
Model Trainning Performance
rmse 1533.3490263894503
MAE 1060.939670363524
r2_score <function r2_score at 0x0000022E9C973820>
